# Strategy Development Notebook

This notebook demonstrates how to develop and test trading strategies using UltSignalTrader.

In [ ]:
# Setup imports
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from src.ultsignaltrader.strategies.base import BaseStrategy, Signal
from src.ultsignaltrader.strategies.ema_crossover import EMACrossoverStrategy
from src.ultsignaltrader.backtests.engine import BacktestEngine
from src.ultsignaltrader.backtests.data_loader import DataLoader

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Generate Sample Data

First, let's create some sample data for testing our strategies.

In [ ]:
def generate_sample_data(days=30, trend='random'):
    """Generate sample OHLCV data."""
    dates = pd.date_range(start='2023-01-01', periods=days*24, freq='1H')
    
    if trend == 'uptrend':
        # Generate uptrending data
        base_price = np.linspace(50000, 60000, len(dates))
    elif trend == 'downtrend':
        # Generate downtrending data
        base_price = np.linspace(60000, 50000, len(dates))
    else:
        # Generate random walk
        returns = np.random.normal(0.0002, 0.01, len(dates))
        base_price = 50000 * (1 + returns).cumprod()
    
    # Add noise
    noise = np.random.normal(0, 100, len(dates))
    close_prices = base_price + noise
    
    # Generate OHLC from close
    data = pd.DataFrame({
        'open': close_prices * (1 + np.random.uniform(-0.001, 0.001, len(dates))),
        'high': close_prices * (1 + np.random.uniform(0, 0.002, len(dates))),
        'low': close_prices * (1 + np.random.uniform(-0.002, 0, len(dates))),
        'close': close_prices,
        'volume': np.random.uniform(100, 200, len(dates))
    }, index=dates)
    
    return data

# Generate different types of data
uptrend_data = generate_sample_data(30, 'uptrend')
downtrend_data = generate_sample_data(30, 'downtrend')
random_data = generate_sample_data(90, 'random')

print(f"Generated {len(random_data)} hours of sample data")

## 2. Visualize Price Data

In [ ]:
def plot_ohlcv(data, title="Price Chart"):
    """Plot OHLCV data."""
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), height_ratios=[3, 1])
    
    # Price chart
    ax1.plot(data.index, data['close'], label='Close', linewidth=2)
    ax1.fill_between(data.index, data['low'], data['high'], alpha=0.2, label='High/Low Range')
    ax1.set_ylabel('Price ($)')
    ax1.set_title(title)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Volume chart
    ax2.bar(data.index, data['volume'], alpha=0.5)
    ax2.set_ylabel('Volume')
    ax2.set_xlabel('Date')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot the data
plot_ohlcv(random_data, "Random Walk Price Data")

## 3. Test EMA Crossover Strategy

In [ ]:
# Initialize strategy
strategy = EMACrossoverStrategy(fast_period=12, slow_period=26, position_size_pct=0.1)

# Calculate indicators
data_with_indicators = strategy.calculate_indicators(random_data)

# Plot price with EMAs
plt.figure(figsize=(14, 7))
plt.plot(data_with_indicators.index, data_with_indicators['close'], 
         label='Close Price', alpha=0.7)
plt.plot(data_with_indicators.index, data_with_indicators['ema_fast'], 
         label=f'Fast EMA ({strategy.fast_period})', linewidth=2)
plt.plot(data_with_indicators.index, data_with_indicators['ema_slow'], 
         label=f'Slow EMA ({strategy.slow_period})', linewidth=2)

plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.title('Price with EMA Indicators')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Run Backtest

In [ ]:
# Initialize backtest engine
engine = BacktestEngine(initial_capital=10000, commission=0.001)

# Run backtest
result = engine.run(strategy, random_data, warmup_period=30)

# Display results
print("\n" + "="*50)
print("BACKTEST RESULTS")
print("="*50)

metrics = result.to_dict()
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key:20}: {value:,.2f}")
    else:
        print(f"{key:20}: {value}")

## 5. Visualize Backtest Results

In [ ]:
# Plot equity curve and drawdown
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Equity curve
ax1.plot(result.equity_curve, label='Portfolio Value', linewidth=2)
ax1.axhline(y=result.initial_capital, color='r', linestyle='--', 
           label='Initial Capital', alpha=0.5)
ax1.set_ylabel('Portfolio Value ($)')
ax1.set_title('Equity Curve')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Calculate and plot drawdown
rolling_max = result.equity_curve.expanding().max()
drawdown = (result.equity_curve - rolling_max) / rolling_max * 100

ax2.fill_between(drawdown.index, drawdown.values, 0, 
               color='red', alpha=0.3, label='Drawdown')
ax2.set_ylabel('Drawdown (%)')
ax2.set_xlabel('Date')
ax2.set_title('Portfolio Drawdown')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Analyze Trade Distribution

In [ ]:
# Analyze trades
if result.trades:
    trades_df = pd.DataFrame([
        {
            'timestamp': t.timestamp,
            'side': t.side,
            'price': t.price,
            'amount': t.amount,
            'value': t.value
        } for t in result.trades
    ])
    
    print(f"\nTotal trades executed: {len(trades_df)}")
    print(f"Buy trades: {len(trades_df[trades_df['side'] == 'buy'])}")
    print(f"Sell trades: {len(trades_df[trades_df['side'] == 'sell'])}")
    
    # Plot trade markers on price chart
    plt.figure(figsize=(14, 7))
    plt.plot(random_data.index, random_data['close'], 
             label='Close Price', alpha=0.5)
    
    # Add trade markers
    buy_trades = trades_df[trades_df['side'] == 'buy']
    sell_trades = trades_df[trades_df['side'] == 'sell']
    
    if not buy_trades.empty:
        plt.scatter(buy_trades['timestamp'], buy_trades['price'], 
                   color='green', marker='^', s=100, label='Buy', zorder=5)
    
    if not sell_trades.empty:
        plt.scatter(sell_trades['timestamp'], sell_trades['price'], 
                   color='red', marker='v', s=100, label='Sell', zorder=5)
    
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.title('Trading Signals on Price Chart')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No trades executed during backtest")

## 7. Parameter Optimization

Let's test different parameter combinations to find optimal settings.

In [ ]:
# Parameter grid
fast_periods = [5, 10, 15]
slow_periods = [20, 30, 40]

results_grid = []

for fast in fast_periods:
    for slow in slow_periods:
        if fast < slow:  # Valid combination
            # Create strategy
            test_strategy = EMACrossoverStrategy(
                fast_period=fast, 
                slow_period=slow,
                position_size_pct=0.1
            )
            
            # Run backtest
            engine.reset()
            test_result = engine.run(test_strategy, random_data, warmup_period=slow+10)
            
            results_grid.append({
                'fast_period': fast,
                'slow_period': slow,
                'total_return_pct': test_result.total_return_pct,
                'sharpe_ratio': test_result.sharpe_ratio,
                'max_drawdown_pct': test_result.max_drawdown_pct,
                'total_trades': test_result.total_trades
            })

# Convert to DataFrame for analysis
optimization_df = pd.DataFrame(results_grid)
optimization_df = optimization_df.sort_values('sharpe_ratio', ascending=False)

print("\nParameter Optimization Results:")
print("="*80)
print(optimization_df.to_string(index=False))

# Best parameters
best_params = optimization_df.iloc[0]
print(f"\nBest parameters: Fast={best_params['fast_period']}, Slow={best_params['slow_period']}")
print(f"Sharpe Ratio: {best_params['sharpe_ratio']:.2f}")

## 8. Create Custom Strategy

Here's an example of how to create a custom RSI-based strategy.

In [ ]:
import ta

class RSIStrategy(BaseStrategy):
    """RSI-based trading strategy."""
    
    def __init__(self, rsi_period=14, oversold=30, overbought=70, position_size_pct=0.1):
        super().__init__(
            name="RSI_Strategy",
            params={
                'rsi_period': rsi_period,
                'oversold': oversold,
                'overbought': overbought,
                'position_size_pct': position_size_pct
            }
        )
        self.rsi_period = rsi_period
        self.oversold = oversold
        self.overbought = overbought
        self.position_size_pct = position_size_pct
    
    def calculate_indicators(self, data):
        """Calculate RSI indicator."""
        df = data.copy()
        
        # Calculate RSI
        df['rsi'] = ta.momentum.RSIIndicator(
            close=df['close'], 
            window=self.rsi_period
        ).rsi()
        
        return df
    
    def generate_signal(self, data):
        """Generate signal based on RSI levels."""
        current_rsi = data['rsi'].iloc[-1]
        prev_rsi = data['rsi'].iloc[-2] if len(data) > 1 else current_rsi
        
        # Buy when RSI crosses above oversold level
        if prev_rsi <= self.oversold and current_rsi > self.oversold:
            return Signal.BUY
        
        # Sell when RSI crosses below overbought level
        elif prev_rsi >= self.overbought and current_rsi < self.overbought:
            return Signal.SELL
        
        return Signal.HOLD
    
    def calculate_position_size(self, signal, current_price, portfolio_value, current_position=None):
        """Calculate position size."""
        if signal == Signal.HOLD:
            return 0.0
        
        if signal == Signal.BUY:
            if current_position and current_position.amount > 0:
                return 0.0
            return (portfolio_value * self.position_size_pct) / current_price
        
        elif signal == Signal.SELL:
            if not current_position or current_position.amount <= 0:
                return 0.0
            return current_position.amount

# Test the RSI strategy
rsi_strategy = RSIStrategy(rsi_period=14, oversold=30, overbought=70)
engine.reset()
rsi_result = engine.run(rsi_strategy, random_data, warmup_period=20)

print(f"\nRSI Strategy Results:")
print(f"Total Return: {rsi_result.total_return_pct:.2f}%")
print(f"Sharpe Ratio: {rsi_result.sharpe_ratio:.2f}")
print(f"Max Drawdown: {rsi_result.max_drawdown_pct:.2f}%")
print(f"Total Trades: {rsi_result.total_trades}")

## 9. Compare Strategies

Let's compare the performance of different strategies.

In [ ]:
# Test multiple strategies
strategies = [
    ('EMA Crossover', EMACrossoverStrategy(fast_period=12, slow_period=26)),
    ('RSI', RSIStrategy(rsi_period=14, oversold=30, overbought=70)),
    ('Buy and Hold', None)  # We'll simulate buy and hold
]

comparison_results = []

for name, strategy in strategies:
    engine.reset()
    
    if name == 'Buy and Hold':
        # Simulate buy and hold
        initial_price = random_data['close'].iloc[0]
        final_price = random_data['close'].iloc[-1]
        total_return_pct = ((final_price - initial_price) / initial_price) * 100
        
        comparison_results.append({
            'Strategy': name,
            'Total Return %': total_return_pct,
            'Sharpe Ratio': 0,
            'Max Drawdown %': 0,
            'Total Trades': 1
        })
    else:
        result = engine.run(strategy, random_data, warmup_period=30)
        comparison_results.append({
            'Strategy': name,
            'Total Return %': result.total_return_pct,
            'Sharpe Ratio': result.sharpe_ratio,
            'Max Drawdown %': result.max_drawdown_pct,
            'Total Trades': result.total_trades
        })

# Display comparison
comparison_df = pd.DataFrame(comparison_results)
print("\nStrategy Comparison:")
print("="*70)
print(comparison_df.to_string(index=False, float_format='%.2f'))

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(comparison_df))
width = 0.35

ax.bar(x - width/2, comparison_df['Total Return %'], width, label='Total Return %')
ax.bar(x + width/2, comparison_df['Sharpe Ratio'], width, label='Sharpe Ratio')

ax.set_xlabel('Strategy')
ax.set_ylabel('Value')
ax.set_title('Strategy Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Strategy'])
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
1. How to generate and visualize sample data
2. Testing the built-in EMA Crossover strategy
3. Running backtests and analyzing results
4. Parameter optimization
5. Creating custom strategies
6. Comparing multiple strategies

You can use this as a template for developing and testing your own trading strategies!